# TABLE OF CONTENTS

* [1. INTRODUCTION](#section-one)
* [2. SETUP](#section-two)
    - [2.1 Draw Packages](#subsection-two-one)
    - [2.2 Import Data](#subsection-two-two)
    - [2.3 Wrangle Data](#subsection-two-three)
* [3. STORY](#section-three)
    - [3.1 Question 1: How bad is the impact of COVID19 on the economy?](#subsection-three-one)
    - [3.2 Question 2: Do new Covid19 cases and deaths correlate with the asset returns in Germany?](#subsection-three-two)
* [4. CONCLUSION](#section-four)
* [5. Bonus Question: Why the stock market rises even when COVID escalates?](#section-five)

<a id="section-one"></a>
# 1. INTRODUCTION

In this notebook, we 
- First explore the effect of COVID19 on the economy in Germany. 
- Then, investigate whether new Covid19 cases and deaths correlate with the asset returns. 
- Finally, take a deep delve into what may go on in the financial market of why the stock market reacts unexpectedly with COVID?


<a id="section-two"></a>
# 2. SETUP

<a id="subsection-two-one"></a>
## 2.1 Draw Packages

In [ ]:
# for numerical analysis
import numpy as np

# to store and process data in dataframe
import pandas as pd

# to interface with operating system
import os

# for basic visualization
import matplotlib.pyplot as plt

# for advanced visualization
import seaborn as sns; sns.set()

# for interactive visualization
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

# for offline interactive visualization
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

# for trendlines
import statsmodels

# data manipulation
from datetime import datetime as dt
from scipy.stats.mstats import winsorize

<a id="subsection-two-two"></a>
## 2.2 Import Data

Econ/Finance: upload the same zipped file from LumiNUS   
* Covid 19: https://www.kaggle.com/imdevskp/corona-virus-report   
* NBER recession indicators: https://www.kaggle.com/stlouisfed/nber-based-recession-indicators-united-states   
* Germany Data https://www.kaggle.com/shaofeisun/germany
* Econfin from bootcamp 1b https://www.kaggle.com/shaofeisun/econfin

In [ ]:
files_G = []

for dirname, _, filenames in os.walk('../input/germany'):
    for filename in filenames:
        files_G.append(os.path.join(dirname, filename))
        
files_G = sorted(files_G)

#Making germany data into a dictionary
series_G = [pd.read_csv(f, na_values=['.']) for f in files_G]
series_name_G = ['10_yr_bond','cpi','DAX','inflation','recession_mon','recession','debt','ltbond','unemployment']
series_dict_G = dict(zip(series_name_G, series_G))

<a id="subsection-two-three"></a>
## 2.3 Wrangle Data
Germany
- DAX Index
- 10 Year Bond
- Long Term Bond (Combined Terms), Similar to 10 year's data
- Inflation indicated by CPI
- Unemployment

In [ ]:
# 1.DAX (Daily and Monthly)
dax = series_dict_G['DAX']

#Convert date into the right format
dax['Date'] = pd.to_datetime(dax['Date'])

#Rename colume from SNP closing data to snp
dax.rename(columns={'Adj Close':'dax'}, inplace=True)

#Calculate % change in stock price
dax['dax_return'] = dax['dax'].pct_change()

# Annualize daily standard deviation * 20 trading days per month (Rolling means to look back X trading days from the recent to the past)
dax['dax_volatility_1m'] = (dax['dax_return'].rolling(20).std())*(20)**(1/2) 

# Annualize daily S.D * 252 trading days per year
dax['dax_volatility_1y'] = (dax['dax_return'].rolling(252).std())*(252)**(1/2) 

#Select he necessary columns
dax = dax[['Date','dax','dax_return','dax_volatility_1m','dax_volatility_1y']]

# Calculate 1-month forward cumulative returns
dax['one_month_forward_dax_return'] = dax['dax_return'][::-1].rolling(window=20, min_periods=1).sum()[::-1]

dax_month = dax.resample('MS',on = 'Date').mean()
dax_month.tail()

plt.plot(dax['Date'], dax['dax'], color = 'r')
plt.title('DAX index again Time')
plt.show()

#There's a quick bounce back of stock price during the covid outbreak
#Possibly due to this is not a typical recession but it's an pendamic-induced recession. Cash is not going to physical business so they are flooded into stock markets. 

In [ ]:
# 2. Germany 10 year bond (MONTHLY)
ten_year_bond = series_dict_G['10_yr_bond']
ten_year_bond['Date'] = pd.to_datetime(ten_year_bond['DATE'])
ten_year_bond.rename(columns={'IRLTLT01DEM156N':'10_yr_bond'}, inplace=True)
ten_year_bond['10_yr_bond_return_1m'] = ten_year_bond['10_yr_bond'].pct_change()
ten_year_bond['10_yr_bond_volatility_1m'] = ten_year_bond['10_yr_bond_return_1m']
#ten_year_bond['10_yr_bond_volatility_1m'] = (ten_year_bond['10_yr_bond_return'].rolling(20).std())*(20)**(1/2)
ten_year_bond['10_yr_bond_volatility_1y'] = (ten_year_bond['10_yr_bond_return_1m'].rolling(12).std())*(12)**(1/2)
ten_year_bond = ten_year_bond[['Date','10_yr_bond','10_yr_bond_return_1m','10_yr_bond_volatility_1m',
                                   '10_yr_bond_volatility_1y']]
ten_year_bond['one_month_forward_10_yr_bond_return'] = ten_year_bond['10_yr_bond_return_1m'][::-1].rolling(window=1, min_periods=1).sum()[::-1]
ten_year_bond.tail()

plt.plot(ten_year_bond['Date'], ten_year_bond['10_yr_bond'], color = 'y')
plt.title('Germany 10-year Bond Return Against Time')
plt.show()

In [ ]:
#3. Germany long term bond (combined terms) (MONTHLY)
ltbond = series_dict_G['ltbond']

ltbond['Date'] = pd.to_datetime(ltbond['DATE'])
ltbond.rename(columns={'IRLTCT01DEM156N':'ltbond'}, inplace=True)
ltbond['ltbond_return_1m'] = ltbond['ltbond'].pct_change()
ltbond['ltbond_volatility_1m'] = ltbond['ltbond_return_1m'] 
#ltbond['ltbond_volatility_1m'] = (ltbond['ltbond_return'].rolling(20).std())*(20)**(1/2)
ltbond['ltbond_volatility_1y'] = (ltbond['ltbond_return_1m'].rolling(12).std())*(12)**(1/2)
ltbond['one_month_forward_inv_grade_bond_return'] = ltbond['ltbond_return_1m'][::-1].rolling(window=1, min_periods=1).sum()[::-1]

ltbond = ltbond[['Date','ltbond','ltbond_return_1m','ltbond_volatility_1m','ltbond_volatility_1y']]

plt.plot(ltbond['Date'], ltbond['ltbond'], color = 'B')
plt.title('Germany Long-Term Bond Return Against Time')
plt.show()


In [ ]:
#4. Inflation in Germany indicated by CPI (MONTHLY)
cpi = series_dict_G['cpi']
cpi['Date'] = pd.to_datetime(cpi['DATE'])
cpi.rename(columns={'DEUCPHPLA01GYM':'cpi'}, inplace=True)
cpi = cpi[['Date','cpi']]
cpi

plt.plot(cpi['Date'], cpi['cpi'], color = 'B')
plt.title('Germany CPI - Inflation Against Time')
plt.show()

In [ ]:
#5. Unemployment  (MONTHLY)
#** This is unemployment rate!
unemployment = series_dict_G['unemployment']
unemployment['Date'] = pd.to_datetime(unemployment['DATE'])
unemployment.rename(columns={'LMUNRRTTDEM156S':'unemployment_rate'}, inplace=True)
unemployment = unemployment[['Date','unemployment_rate']]

plt.plot(unemployment['Date'], unemployment['unemployment_rate'], color = 'g')
plt.show()

In [ ]:
#6. Germany's Debt Position 
#* The data is not sufficient as it's only until 2017
#This is an indicator of economic conditions, not an asset class

#ger_debt = series_dict_G['debt']
#ger_debt['Date'] = pd.to_datetime(ger_debt['DATE'])
#ger_debt.rename(columns={'GGGDTADEA188N':'debt_percent'}, inplace=True)
#ger_debt = ger_debt[['Date','debt_percent']]

#plt.plot(ger_debt['Date'], ger_debt['debt_percent'])
#plt.show()

In [ ]:
# Import datasets with Pandas method read_csv (Daily and Monthly, but only monthly is used)

#import monthly recession indicator (1,0)
recession_indicator_month = series_dict_G['recession_mon']

#Convert data types and rename columns
recession_indicator_month["Date"] = pd.to_datetime(recession_indicator_month["DATE"], format = '%d/%m/%Y')
recession_indicator_month["Recession"] = recession_indicator_month["DEUREC"].astype('bool')

# Subset data columns
recession_indicator_month = recession_indicator_month[["Date","Recession"]]
recession_indicator_month.tail()


In [ ]:
# Merge datasets together (All monthly datasets)
asset_classes = [cpi,ten_year_bond,ltbond,unemployment]

baseline = pd.merge(dax_month,recession_indicator_month,how='left',left_on='Date', right_on = 'Date')

for asset_class in asset_classes:
    baseline = pd.merge(baseline,asset_class,how='left',left_on='Date', right_on = 'Date')

#Backfilling missing values, for covid outbreak, assume recession YES
baseline.loc[baseline.Date >= '2020-03-01', "Recession"] = 1
baseline["Recession"] = baseline["Recession"].fillna(0).astype(bool)

baseline.info()
baseline.tail(10)

In [ ]:
# Grouped by Country, Filter Germany and Resample into Monthly Data
# =======================

full_grouped = pd.read_csv('../input/corona-virus-report/full_grouped.csv')
full_grouped.info()

# Convert Date from Dtype "Object" (or String) to Dtype "Datetime", filtering for Germany
full_grouped['Date'] = pd.to_datetime(full_grouped['Date'])
g_covid = full_grouped[full_grouped['Country/Region']=="Germany"]
g_covid_month = g_covid.resample('MS', on = 'Date').mean()
g_covid_month.tail(10)



In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#2020 covid19 period in Germany with assets
baseline2020 = baseline[baseline['Date'] >= '2019-12-01']
baseline2020 = pd.merge(baseline2020,g_covid_month, how='left', on='Date')
baseline2020['New cases'] = baseline2020['New cases'].fillna(0)

baseline2020.tail(10)

<a id="section-three"></a>
# 3. STORY

<a id="subsection-three-one"></a>
## 3.1 Question 1: What is the impact of COVID19 on the Germany stock market (DAX)?

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces to create subplots
fig.add_trace(
    go.Scatter(x=baseline2020['Date'], y=baseline2020['dax'], name = 'DAX'),  
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=baseline2020['Date'], y=baseline2020['New cases'], name = 'New COVID19 Cases'), 
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="DAX and New COVID19 Cases in Germany"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>DAX</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>New COVID19 Cases</b>", secondary_y=True)

fig.show()

In [ ]:
def plot_chart(series):
    fig = px.scatter(baseline[baseline[series].notnull()], x="Date", y=series, color='Recession', width=1000)
    fig.update_traces(mode='markers', marker_size=4)
    fig.update_layout(title=series, xaxis_title="", yaxis_title="")
    fig.show()

In [ ]:
baseline['dax_volatility_1m'].describe()

The historical standard deviation of monthly return for DAX is 2.8%.

In [ ]:
print("The worst single-month return in 2020 is ", str(round(abs(baseline2020['dax_return'].min()/baseline['dax_return'].std()),2)), 
      " X standard deviations of DAX historical returns!")

Let's put this into perspective with a histogram of DAX past returns
The DAX volatility is generally less as it's a extremely mature market in EU

In [ ]:
# Output the range of DAX historical daily returns from 1928-01-03 to 2020-07-01
print("DAX historical monthly returns from " + str(baseline[baseline['dax_return'].notnull()]['Date'].min().date()) + ' to '
       + str(baseline[baseline['dax_return'].notnull()]['Date'].max().date()))

fig = px.histogram(baseline, x="dax_return")
fig.show()

What is the impact of Covid19 on employment in Germany?

In [ ]:
plot_chart("unemployment_rate")

Umemployment rate rose sharply during the covid period in 2020

In [ ]:
print("This is ", str(round(abs(baseline['unemployment_rate'].min()/baseline['unemployment_rate'].std()),2)), 
      " X standard deviations of the historical monthly change in unemployment!")

The impact of COVID19 on the economy has been pretty nasty! Why is the market rebounding so fast? The V-shaped recovery in the DAX does not seem to correspond with the economic recovery. 
Before we answer this, let's explore the relationship between COVID19 and the financial market in the next question.

<a id="subsection-three-two"></a>
## 3.2 Question 2: Do new Covid19 cases and deaths correlate with asset returns in Germany?

In [ ]:
sns.jointplot(x = 'New cases', y = 'dax', data = baseline2020, kind='reg')

It makes sense to have Negative correlation when Covid escalates, DAX index falls

In [ ]:
sns.jointplot(x = 'New deaths', y = 'dax_return', data = baseline2020, kind='reg')

Positive correlation? The number of deaths does not necessarily coincides with the new cases. It's the number of new cases that truly affects the economy.

In [ ]:
# Draw scatter of asset returns during Covid19 pandemic
baseline2020.info()
baseline_returns = baseline2020[["dax_return", "10_yr_bond_return_1m", "ltbond_return_1m", "New deaths", "New cases"]]

sns.pairplot(baseline_returns)

Unobservable relations are seen between the different assets as only monthly data is available. Let's take a look into the heatmap below

In [ ]:
# Draw heatmap of correlation strength across asset classes (returns and volatilities) and Covid19 new cases and deaths during the pandemic period 
baseline_corr = baseline2020[['dax_return', 'dax_volatility_1y', '10_yr_bond_return_1m', '10_yr_bond_volatility_1y',
                         'ltbond_return_1m', 'ltbond_volatility_1y','New deaths', 'New cases']].corr()

fig, ax = plt.subplots(figsize=(8,5)) 
sns.heatmap(baseline_corr, annot=True, ax = ax)

<a id="section-four"></a>
# 4. CONCLUSION

The effect of Covid19 pandemic on the Germany economy has been unprecendented. One would expect the financial markets in Germany to mirror the state of the economy. However, the financial markets have been on a divergent path. After the cliff in the first quarter of 2020, the financial markets have been on a steady rebounce, closing in to the pre-pandemic levels. 

We investigate the plausible reasons for such divergence: helicopter, optimism of the prospect of strong economic recovery, and FOMO effect that every other country in the world is facing. For germany, we are expecting a fer other reasons that's stated in part 5 below.

# 5.Bonus
Why does the stock market keep bouncing back dispite the escalating covid situation?

Some of the insight shows that the steady rise of stock market could be due to the reasons below,

1. Easing Fiscal Policy
> Governments around the world are trying to stimulate the economy when the covid hits. However, most of the physical busiensses are affected heavily so most of the fund goes into the stock market during the stay-home period.
> 
2. Capital has no where to go
> As businesses are being shut down and carry tons of tisk of default during the pandemic, capitals are looking for alternative investments in 2020. Lots of capitals are flowing in to the stock market to fight for a slice of pie as the market is can be easily manipulated to reap from the robinson investors. The index in many countries, being developed or developing countries , have reached unprecendented high and are like a big bubble that's about to burst anytime.
> 
3. Government Policy to stablise people's mind
> Stock market index is actually a good indicator of how the economy is doing. Some government use different measures to spur up the stock index to reassure local and international investors that the economy in its country is bouncing back well and is able to handle the pandemic. That's another reason why the stock market is doing well unexpectedly.